In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# Mount Google Drive to access dataset
from google.colab import drive
drive.mount('/content/drive')

# Set up directory paths
base_dir = '/content/drive/MyDrive/Dataset Hasil Split 80-20_100x100_0'
h5_dir = '/content/drive/MyDrive/Capstone Project ML/CNN_Model3.h5'
train_dir = os.path.join(base_dir, 'training')
test_dir = os.path.join(base_dir, 'validation')

Mounted at /content/drive


In [3]:
# # Define the model
# model = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(100,100,3)),
#     MaxPooling2D((2,2)),
#     Conv2D(64, (3,3), activation='relu'),
#     MaxPooling2D((2,2)),
#     Conv2D(128, (3,3), activation='relu'),
#     MaxPooling2D((2,2)),
#     Conv2D(256, (3,3), activation='relu'),
#     MaxPooling2D((2,2)),
#     Flatten(),
#     Dense(512, activation='relu'),
#     Dropout(0.2),
#     Dense(256, activation='relu'),
#     Dropout(0.2),
#     Dense(5, activation='softmax')
# ])

# # Compile the model
# opt = Adam(learning_rate=0.0001)
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [4]:
# # Load the pre-trained model from the .h5 file
# model = tf.keras.models.load_model(h5_dir)

In [5]:
# Set up data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   brightness_range=(0.8,1.2),
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range=20,
                                 brightness_range=(0.8,1.2),
                                 horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(100, 100),
                                                    batch_size=16,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(test_dir,
                                                target_size=(100, 100),
                                                batch_size=16,
                                                class_mode='categorical')

FileNotFoundError: ignored

In [ ]:
# Set up checkpoint callback
checkpoint = ModelCheckpoint("CNN_Model3.h5", 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

In [ ]:
# Train the model
history = model.fit(train_generator,
                    epochs=40,
                    validation_data=val_generator,
                    callbacks=[checkpoint])

In [ ]:
# Re-evaluate the model
loss, acc = model.evaluate(val_generator, verbose=2)
print("Accuracy: {:5.2f}%".format(100 * acc))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

In [ ]:
# Create a figure with two subplots
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Plotting accuracy
ax[0].plot(epochs, acc, 'r', linewidth=2, label='Training Accuracy')
ax[0].plot(epochs, val_acc, 'b', linewidth=2, label='Validation Accuracy')
ax[0].set_title('Training and Validation Accuracy', fontsize=16)
ax[0].set_xlabel('Epoch', fontsize=14)
ax[0].set_ylabel('Accuracy', fontsize=14)
ax[0].legend(loc='lower right', fontsize=12)
ax[0].grid()
ax[0].tick_params(axis='both', labelsize=12)

# Plotting loss
ax[1].plot(epochs, loss, 'r', linewidth=2, label='Training Loss')
ax[1].plot(epochs, val_loss, 'b', linewidth=2, label='Validation Loss')
ax[1].set_title('Training and Validation Loss', fontsize=16)
ax[1].set_xlabel('Epoch', fontsize=14)
ax[1].set_ylabel('Loss', fontsize=14)
ax[1].legend(loc='upper right', fontsize=12)
ax[1].grid()
ax[1].tick_params(axis='both', labelsize=12)

plt.show()

In [ ]:
# Get the predicted labels for all images in the cross validation set
pred = model.predict(val_generator)
predicted_labels = np.argmax(pred, axis=1)

# Get the true labels for all images in the cross validation set
true_labels = val_generator.classes

# Get the filenames of the images in the cross validation set
filenames = val_generator.filenames

# Get the indices of the misclassified images
misclassified_indices = np.where(predicted_labels != true_labels)[0]

# Get the misclassified images and their true and predicted labels
misclassified_images = []
misclassified_true_labels = []
misclassified_predicted_labels = []

for i in misclassified_indices:
    misclassified_images.append(filenames[i])
    misclassified_true_labels.append(true_labels[i])
    misclassified_predicted_labels.append(predicted_labels[i])

# Print the misclassified images and their true and predicted labels
print("Misclassified Images:")
for i in range(len(misclassified_images)):
    print("Image:", misclassified_images[i], "True Label:", misclassified_true_labels[i], "Predicted Label:", misclassified_predicted_labels[i])

In [ ]:
# Display some of the misclassified images
num_images_to_display = 36
fig, axes = plt.subplots(6, 6, figsize=(12, 12))
axes = axes.ravel()

# Map class indices to class names
class_names = {v: k for k, v in val_generator.class_indices.items()}

for i in range(num_images_to_display):
    img = Image.open(os.path.join(test_dir, misclassified_images[i]))
    true_label = class_names[misclassified_true_labels[i]]
    predicted_label = class_names[misclassified_predicted_labels[i]]
    axes[i].imshow(img)
    axes[i].set_title("True: {}\nPredicted: {}".format(true_label, predicted_label), fontsize=10)
    axes[i].axis('off')
plt.tight_layout()
plt.show()